# k-nearest neighbour

In [ ]:
import numpy as np
import cv2

## training the model

### preparing the dataset

In [ ]:
dataset = cv2.imread('images/digits_dataset.png')
gray = cv2.cvtColor(dataset, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(dataset)

print('image shape -', gray.shape)
# cv2.imshow('image', dataset)
# cv2.waitKey(0)

cells = []
for row in np.vsplit(gray, 50):
    row_shape = row.shape
    row_example = row
    cells.append(np.hsplit(row, 100))


print('row shape -', row_shape)
# cv2.imshow('image', row_example)
# cv2.waitKey(0)

cells = np.array(cells)

cell_example = cells[-1][0]

print('cell shape -', cell_example.shape)
# cv2.imshow('image', cell_example)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

print('cells array shape -', cells.shape)

In [ ]:
# 70% for tain and 30% for test
train = cells[:, :70].reshape(-1, 400).astype(np.float32)
test = cells[:,70:100].reshape(-1, 400).astype(np.float32)

print('train shape -', train.shape)
print('test shape -', test.shape)

### labeling

In [ ]:
digits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_labels = np.repeat(digits, 350)[:, np.newaxis]
test_labels = np.repeat(digits, 150)[:, np.newaxis]

print('train labels shape -', train_labels.shape)
print('test labels shape -', test_labels.shape)

### training and evaluating

In [ ]:
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, results, neighbours, dist = knn.findNearest(test, 3)

correct = 0
for i, result in enumerate(results):
    if result == test_labels[i]:
        correct += 1

accuracy = correct * (100.0 / results.size)
print("accuracy: %.2f" % accuracy + "%")

## using the model

### preparing the image

In [ ]:
image = cv2.imread('images/digits.png')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
canny = cv2.Canny(blurred, 30, 150)
# cv2.imshow('image', blurred)
# cv2.waitKey(0)

contours, _ = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# https://answers.opencv.org/question/179510/how-can-i-sort-the-contours-from-left-to-right-and-top-to-bottom/
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[1])

new_contours = []
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 20:
        new_contours.append(contour)

contours = np.array(new_contours)

outlined = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 1)
# cv2.imshow('image', outlined)
# cv2.waitKey(0)

# cv2.destroyAllWindows()

In [ ]:
numbers = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    roi = blurred[y-5:y + h+5, x-5:x + w+5]

    _, roi = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY_INV)

    # transforming the roi into a square
    # https://stackoverflow.com/questions/45646201/how-do-i-make-rectangular-image-squared-using-opencv-and-python
    height, width = roi.shape
    x = height if height > width else width
    y = height if height > width else width
    square= np.zeros((x, y), np.uint8)
    square[int((y-height)/2):int(y-(y-height)/2), int((x-width)/2):int(x-(x-width)/2)] = roi

    # cv2.imshow('square', square)
    # cv2.imshow('roi', roi)
    # cv2.waitKey(0)

    # turning the image into the input form for the model
    square = cv2.resize(square, (20, 20), interpolation=cv2.INTER_LANCZOS4)
    _, square = cv2.threshold(square, 20, 255, cv2.THRESH_BINARY)

    # cv2.imshow('square', square)
    # cv2.waitKey(0)

    number = square.reshape((1, 400)).astype(np.float32)
    numbers.append(number)
# cv2.destroyAllWindows()

### getting the results

In [ ]:
result = []
for number in numbers:
    ret, results, neighbours, dist = knn.findNearest(number, 2)
    result.append(int(results))

### plotting the results

In [ ]:
numbers = image.copy()
final_number = ""
for i, contour in enumerate(contours):
    number = str(result[i])
    final_number += number
    x, y, w, h = cv2.boundingRect(contour)
    cv2.putText(numbers, number, (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    

cv2.imshow('image', numbers)
cv2.waitKey(0)
cv2.destroyAllWindows()
print('number:',final_number)